In [41]:
from threading import Thread
from threading import Lock
from random import randint
import numpy as np
import time
import sys

**Programa 1**

In [43]:
def sprint(text):
    sys.stdout.write("%s\n" % text)
class competidor(Thread):
    def __init__(self,nombre):
        super().__init__()
        self.posicion = 0
        self.nombre = nombre
        self.lock = Lock()
    def turno(self,random,probabilidades,tiempo):
        probabilidadInicial = 0
        for t in probabilidades:
            #Calcula si el numero generado está en el rango actual
            if probabilidadInicial <  random <= probabilidadInicial + t[0]:
                self.lock.acquire()
                try:
                    time.sleep(.11)
                    self.posicion += t[2]
                    if self.posicion < 0:
                        self.posicion = 0
                    elif self.posicion > 70:
                        self.posicion = 70
                finally:
                    sprint("Turno "+str(tiempo)+":"+str(' '*(self.posicion-1))+self.nombre[0])
                    self.lock.release()
                break
            probabilidadInicial += t[0]
    def getPos(self):
        self.lock.acquire()
        p = 0
        try:
            p = self.posicion
        finally:
            self.lock.release()
            return p
class tortuga(competidor):
    def __init__(self):
        super().__init__('Tortuga')
    def turno(self,random,t):
        probabilidades = [(50,'Avance rápido',3),(20,'Resbaló',-6),(30,'Avance lento',1)]
        super().turno(random,probabilidades,t)
class liebre(competidor):
    def __init__(self):
        super().__init__('Liebre')
    def turno(self,random,t):
        probabilidades = [(20,'Duerme',0),(20,'Gran salto',9),(10,'Resbalón grande',-12),(30,'Pequeño salto',1),(20,'Resbalón pequeño',-2)]
        super().turno(random,probabilidades,t)
if __name__ == '__main__':
    competidores = []
    competidores.append(tortuga())
    competidores.append(liebre())
    for c in competidores:
        c.start()
    continuarCarrera = True
    winner = []
    turno = 1
    while(continuarCarrera):
        for c in competidores:
            r = randint(1, 100)
            c.turno(r,turno)
        for c in competidores:
            if c.getPos() == 70:
                continuarCarrera = False
                winner.append(c)
        turno += 1
    resultString = ''
    if len(winner) == 1:
        resultString = 'El ganador es: '+winner[0].nombre
    else:
        resultString = 'Se declara un empate'
    print(resultString)
    for c in competidores:
        c.join()

Turno 1:T
Turno 1:L
Turno 2:T
Turno 2:L
Turno 3:  T
Turno 3:        L
Turno 4:   T
Turno 4:        L
Turno 5:T
Turno 5:         L
Turno 6:T
Turno 6:          L
Turno 7: T
Turno 7:                   L
Turno 8:    T
Turno 8:                   L
Turno 9:       T
Turno 9:                 L
Turno 10:        T
Turno 10:                  L
Turno 11:           T
Turno 11:                  L
Turno 12:              T
Turno 12:                   L
Turno 13:               T
Turno 13:       L
Turno 14:                  T
Turno 14:        L
Turno 15:            T
Turno 15:         L
Turno 16:             T
Turno 16:          L
Turno 17:              T
Turno 17:          L
Turno 18:               T
Turno 18:        L
Turno 19:                T
Turno 19:         L
Turno 20:                 T
Turno 20:         L
Turno 21:           T
Turno 21:          L
Turno 22:     T
Turno 22:           L
Turno 23:T
Turno 23:                    L
Turno 24:  T
Turno 24:                    L
Turno 25:   T
Turno 25:   

**Programa 2: Multiplicación de matrices de gran dimensión**

In [82]:
matriz1 = np.random.randint(10, size=(1000, 1200))
matriz2 = np.random.randint(10, size=(1200, 1000))
columnas1 = np.size(matriz1,1)
columnas2 = np.size(matriz2,1)
renglones1 = np.size(matriz1,0)
renglones2 = np.size(matriz2,0)
#Version secuencial
def multSec(m1,m2,resultado):
    tiempoInicio = time.time()
    for y in range(renglones1):
        for x in range(columnas2):
            for z in range(renglones2):
                resultado[y][x] += m1[y][z]*m2[z][x]
    sprint("El tiempo del método secuencial es :"+str(time.time()-tiempoInicio))
#Divide el problema en cuatro cuadrantes
def multConcCuatro(m1,m2,aX,aY):
    global resultado
    lY = np.ceil(renglones1/2).astype(int)
    lX = np.ceil(columnas2/2).astype(int)
    for y in range(lY*aY,lY+(aY*lY)):
        for x in range(lX*aX,lX+(aX*lX)):
            for z in range(renglones2):
                resultado[y][x] += m1[y][z]*m2[z][x]
if __name__ == '__main__':
    resultado = np.zeros((columnas2,renglones1))
    multSec(matriz1,matriz2,resultado)
    print(resultado)
    resultado = np.zeros((columnas2,renglones1))
    threads = []
    for x in range(2):
        for y in range(2):
            threads.append(Thread(target=multConcCuatro, args=(matriz1,matriz2,x,y,)))
    tiempoInicio = time.time()
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    print("El tiempo del método concurrente es :"+str(time.time()-tiempoInicio))
    print(resultado)

El tiempo del método secuencial es :1078.017989397049
[[24284. 24285. 24185. ... 24538. 23848. 24310.]
 [24773. 24146. 24651. ... 25184. 24377. 24785.]
 [23419. 23850. 23157. ... 24712. 22452. 23980.]
 ...
 [24222. 24389. 24506. ... 26039. 24307. 25603.]
 [23854. 24647. 24294. ... 25198. 24023. 24262.]
 [24098. 23409. 23662. ... 24777. 23195. 24444.]]
El tiempo del método concurrente es :1158.1521155834198
[[24284. 24285. 24185. ... 24538. 23848. 24310.]
 [24773. 24146. 24651. ... 25184. 24377. 24785.]
 [23419. 23850. 23157. ... 24712. 22452. 23980.]
 ...
 [24222. 24389. 24506. ... 26039. 24307. 25603.]
 [23854. 24647. 24294. ... 25198. 24023. 24262.]
 [24098. 23409. 23662. ... 24777. 23195. 24444.]]
